In [12]:
from transformers import AutoModelForSeq2SeqLM, AutoModel, AutoTokenizer
import nltk
import json
import pandas as pd
from datasets import *
import numpy as np
import torch

nltk.download('punkt')
SEED = 42

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
split = "test"

In [14]:
rouge = load_metric("rouge")
bert = load_metric("bertscore")

In [15]:
def compute_rouge(decoded_preds, decoded_labels, prediction_lens):
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

## T5 small

In [16]:
model_checkpoint = "cjvt/t5-sl-small"
model_name = model_checkpoint.split("/")[-1]
data_path = f"../../data/results/prompt_reply_pairs_1_generated_{split}_{model_name}.csv"

data = pd.read_csv(data_path, sep=";")
# display(data)

decoded_labels = data["reply"].to_list()
decoded_preds = data["generated"].to_list()
prediction_lens = data["token_prediction_len"].to_list()
quantitative_data = pd.DataFrame({"decoded_preds": data["generated"], "decoded_labels": data["reply"], "prediction_lens": data["token_prediction_len"]})
# display(quantitative_data)

print(model_name.upper())

# ROUGE
result_rouge = compute_rouge(decoded_preds, decoded_labels, prediction_lens)
print(result_rouge)

# BERTSCORE
result_bert = bert.compute(predictions=decoded_preds, references=decoded_labels, lang="sl")
result_bert = {"Precision": np.array(result_bert["precision"]).mean(), "Recall": np.array(result_bert["recall"]).mean(), "F1": np.array(result_bert["f1"]).mean()}
print(result_bert)

T5-SL-SMALL
{'rouge1': 0.3373, 'rouge2': 0.0647, 'rougeL': 0.2434, 'rougeLsum': 0.3086, 'gen_len': 3.4974}
{'Precision': 0.624625362267861, 'Recall': 0.4409378152504945, 'F1': 0.5145365310693398}


## T5 small finetuned pairs

In [17]:
model_checkpoint = "ls1906/t5-sl-small-finetuned-assistant"
model_name = model_checkpoint.split("/")[-1]
data_path = f"../../data/results/prompt_reply_pairs_1_generated_{split}_{model_name}.csv"

data = pd.read_csv(data_path, sep=";")
# display(data)

decoded_labels = data["reply"].to_list()
decoded_preds = data["generated"].to_list()
prediction_lens = data["token_prediction_len"].to_list()
quantitative_data = pd.DataFrame({"decoded_preds": data["generated"], "decoded_labels": data["reply"], "prediction_lens": data["token_prediction_len"]})
# display(quantitative_data)

print(model_name.upper())

# ROUGE
result_rouge = compute_rouge(decoded_preds, decoded_labels, prediction_lens)
print(result_rouge)

# BERTSCORE
result_bert = bert.compute(predictions=decoded_preds, references=decoded_labels, lang="sl")
result_bert = {"Precision": np.array(result_bert["precision"]).mean(), "Recall": np.array(result_bert["recall"]).mean(), "F1": np.array(result_bert["f1"]).mean()}
print(result_bert)

T5-SL-SMALL-FINETUNED-ASSISTANT
{'rouge1': 17.2245, 'rouge2': 4.0757, 'rougeL': 11.48, 'rougeLsum': 15.7309, 'gen_len': 73.3327}
{'Precision': 0.6697491393517225, 'Recall': 0.6337999124588111, 'F1': 0.649464008190693}
